In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

from ipywidgets import IntProgress
from IPython.display import display

In [2]:
url = "https://www.ebay.com/b/Boys-Short-Sleeve-Sleeve-Tops-T-Shirts-Sizes-4-Up/175521/bn_4278610?rt=nc&LH_ItemCondition=1000&LH_BIN=1&LH_PrefLoc=3&_pgn=1"
r = requests.get(url)
soup = BeautifulSoup(r.content)

In [3]:
images = soup.find_all('img')[1:]
images

[<img alt="Polo Ralph Lauren Boys V-neck Classic T-shirt Pony Logo Tee Kids Striped Collar" class="s-item__image-img" onload='SITE_SPEED.ATF_TIMER.measure(this); if (performance &amp;&amp; performance.mark) { performance.mark("first-meaningful-paint"); }' src="https://i.ebayimg.com/thumbs/images/m/mPk_PdWCI70ziT-cYO3ujaA/s-l225.jpg"/>,
 <img alt="NWT Under Armour Boy's heatgear Big Logo 1290097 Loose Fit T-Shirt" class="s-item__image-img" onload="SITE_SPEED.ATF_TIMER.measure(this)" src="https://i.ebayimg.com/thumbs/images/m/mocbpz9cC9idlBcU6UxXj_A/s-l225.jpg"/>,
 <img alt="Tommy Hilfiger Kids T-Shirt Big Boys Solid Crew Neck Tee Children Size 4-18 New" class="s-item__image-img" onload="SITE_SPEED.ATF_TIMER.measure(this)" src="https://i.ebayimg.com/thumbs/images/m/mymwuucjBVuu2IjMcivCm7g/s-l225.jpg"/>,
 <img alt="Polo Ralph Lauren Boys Polo Shirt Classic Mesh Short Sleeve Pony Logo Kids Top" class="s-item__image-img" onload="SITE_SPEED.ATF_TIMER.measure(this)" src="https://i.ebayimg.com

In [4]:
image_titles = [img['alt'] for img in images]
image_titles

['Polo Ralph Lauren Boys V-neck Classic T-shirt Pony Logo Tee Kids Striped Collar',
 "NWT Under Armour Boy's heatgear Big Logo 1290097 Loose Fit T-Shirt",
 'Tommy Hilfiger Kids T-Shirt Big Boys Solid Crew Neck Tee Children Size 4-18 New',
 'Polo Ralph Lauren Boys Polo Shirt Classic Mesh Short Sleeve Pony Logo Kids Top',
 'Polo Ralph Lauren Boys Big Pony Polo Shirt Classic Fit Kids Mesh School New Nwt',
 'SHREK Youth Boys Kids Short Sleeve T-Shirt BLACK HEATHER SHREK MUGSHOT',
 "NIB ~ Kelly's Kids Blue/White Stripe Raspberry Lobster Camp Shirt Boy's 6-7 Year",
 'Tommy Hilfiger Children Big Boy Kid Short Sleeve Logo Pique Polo Shirt - $0 Ship',
 'NWT POLO RALPH LAUREN BOYS COLOR BLOCKED RED WHITE BIG PONY RUGBY SHIRT',
 'Tommy Hilfiger Children Big Boy Crew Neck Logo Tee T-Shirt - Free $0 Shipping',
 'NWT Ralph Lauren Boys SS Cotton Madras Plaid Button Down Shirt 8 14/16 18/20 NEW',
 'NWT Ralph Lauren Boys SS Big Pony Classic Solid Mesh Polo Shirt Sz 5 6 7 NEW $35',
 'NWT Ralph Lauren Bo

In [5]:
image_files_src = [img['src'] for img in images]
image_files_datasrc = [img.get('data-src', None) for img in images]
image_files = [src if datasrc is None else datasrc for src, datasrc in zip(image_files_src, image_files_datasrc)]
image_files

['https://i.ebayimg.com/thumbs/images/m/mPk_PdWCI70ziT-cYO3ujaA/s-l225.jpg',
 'https://i.ebayimg.com/thumbs/images/m/mocbpz9cC9idlBcU6UxXj_A/s-l225.jpg',
 'https://i.ebayimg.com/thumbs/images/m/mymwuucjBVuu2IjMcivCm7g/s-l225.jpg',
 'https://i.ebayimg.com/thumbs/images/m/mbAW78EZTb7qwbMe51QkaBQ/s-l225.jpg',
 'https://i.ebayimg.com/thumbs/images/m/mpA3cslyf1wxmuxXZMDXYRQ/s-l225.jpg',
 'https://i.ebayimg.com/thumbs/images/m/mEfuPaN8hGMxyIZyMKjKiyA/s-l225.jpg',
 'https://i.ebayimg.com/thumbs/images/g/jWEAAOSwuhhXVV1C/s-l225.jpg',
 'https://i.ebayimg.com/thumbs/images/m/mU_3OaCxPH8eHRoq_UsIMTg/s-l225.jpg',
 'https://i.ebayimg.com/thumbs/images/m/muHanFhhiWP55_OjurfiSEg/s-l225.jpg',
 'https://i.ebayimg.com/thumbs/images/m/mB-Wenbs9m8w0m9fFiaWpQA/s-l225.jpg',
 'https://i.ebayimg.com/thumbs/images/m/m5zYvrmnYxZxDQM3dWkJQeA/s-l225.jpg',
 'https://i.ebayimg.com/thumbs/images/m/mHF_deSH9l-AzQfFQFPpIyg/s-l225.jpg',
 'https://i.ebayimg.com/thumbs/images/m/mKbBoVF-sl80b3EMOvqjPTA/s-l225.jpg',
 'http

In [6]:
def parse_price(price_element):
    try:
        price = float(price_element.get_text().split(' ')[1])
    except:
        price = None
    return price
price_elements = soup.find_all('span', class_ = 's-item__price')

prices = [parse_price(price_e) for price_e in price_elements]

In [7]:
assert len(prices) == len(images)

In [8]:
def download_image(url, title, file_name):
    try:
        response = requests.get(url)    
    except:
        return '', ''
    with open(file_name, "wb") as file:
        file.write(response.content)
    return title, file_name

In [9]:
download_image(image_files[0], image_titles[0], 'test.jpg')

('Polo Ralph Lauren Boys V-neck Classic T-shirt Pony Logo Tee Kids Striped Collar',
 'test.jpg')

In [10]:
download_image('aaa', 'bbb', 'error.jpg')

('', '')

In [11]:
images_data = {'title': {}, 'file_name': {}, 'price': {}}

f = IntProgress(min=0, max=len(images)) # instantiate the bar
display(f) # display the bar

for i in range(len(images)):
    title, file_name = download_image(image_files[i], image_titles[i], './boys/' + str(i) + '.jpg')
    images_data['title'][i] = title
    images_data['file_name'][i] = file_name
    images_data['price'][i] = prices[i]
    f.value += 1
    
pd.DataFrame(images_data)
    

IntProgress(value=0, max=48)

,title,file_name,price
0,Polo Ralph Lauren Boys V-neck Classic T-shirt ...,./boys/0.jpg,70.96
1,NWT Under Armour Boy's heatgear Big Logo 12900...,./boys/1.jpg,67.22
2,Tommy Hilfiger Kids T-Shirt Big Boys Solid Cre...,./boys/2.jpg,74.70
3,Polo Ralph Lauren Boys Polo Shirt Classic Mesh...,./boys/3.jpg,81.42
4,Polo Ralph Lauren Boys Big Pony Polo Shirt Cla...,./boys/4.jpg,130.75
5,SHREK Youth Boys Kids Short Sleeve T-Shirt BLA...,./boys/5.jpg,67.22
6,NIB ~ Kelly's Kids Blue/White Stripe Raspberry...,./boys/6.jpg,78.43
7,Tommy Hilfiger Children Big Boy Kid Short Slee...,./boys/7.jpg,149.43
8,NWT POLO RALPH LAUREN BOYS COLOR BLOCKED RED W...,./boys/8.jpg,93.38
9,Tommy Hilfiger Children Big Boy Crew Neck Logo...,./boys/9.jpg,112.06


In [12]:
boys_url = 'https://www.ebay.com/b/Boys-Short-Sleeve-Sleeve-Tops-T-Shirts-Sizes-4-Up/175521/bn_4278610?rt=nc&LH_ItemCondition=1000&LH_BIN=1&LH_PrefLoc=3&_pgn='
max_pages = 400
boys_items_data = {'title': {}, 'file_id': {}, 'price': {}}
f = IntProgress(min = 0, max = max_pages)
display(f)
all_items_counter = 0

for page_num in range(max_pages):
    url = boys_url + str(page_num)
    try:
        r = requests.get(url)
    except:
        print('Stopped at page: ' + page_num)
        break
    soup = BeautifulSoup(r.content)
    images = soup.find_all('img')[1:]
    image_titles = [img['alt'] for img in images]
    image_files_src = [img['src'] for img in images]
    image_files_datasrc = [img.get('data-src', None) for img in images]
    image_files = [src if datasrc is None else datasrc for src, datasrc in zip(image_files_src, image_files_datasrc)]
    
    price_elements = soup.find_all('span', class_ = 's-item__price')
    prices = [parse_price(price_e) for price_e in price_elements]
    try:
        assert len(prices) == len(images)
    except:
        print('Found unequal number of prices in page_num % d' % page_num)
        prices = [None] * len(images)
        
    for i in range(len(images)):
        title, file_name = download_image(image_files[i], image_titles[i], './boys/' + str(all_items_counter + i) + '.jpg')
        boys_items_data['title'][all_items_counter + i] = title
        boys_items_data['file_id'][all_items_counter + i] = all_items_counter + i
        boys_items_data['price'][all_items_counter + i] = prices[i]
    all_items_counter += len(images)
    f.value += 1

IntProgress(value=0, max=400)

In [13]:
boys_items_data = pd.DataFrame(boys_items_data)
boys_items_data = boys_items_data[boys_items_data['title'] != '']
print(boys_items_data.shape)

(17184, 3)


In [14]:
boys_items_data = pd.DataFrame(boys_items_data).drop_duplicates('title')
print(boys_items_data.shape)

(12515, 3)


In [15]:
boys_items_data.to_csv('boys_items_data.csv', index_label = 'id')
print('Final boys number of images: %d' % boys_items_data.shape[0])

Final boys number of images: 12515


In [16]:
girls_url = 'https://www.ebay.com/b/Girls-Short-Sleeve-Sleeve-Tops-T-Shirts-Sizes-4-Up/175529/bn_4741026?rt=nc&LH_ItemCondition=1000&LH_BIN=1&LH_PrefLoc=3&_pgn='
max_pages = 400
girls_items_data = {'title': {}, 'file_id': {}, 'price': {}}
f = IntProgress(min = 0, max = max_pages)
display(f)
all_items_counter = 0

for page_num in range(max_pages):
    url = girls_url + str(page_num)
    try:
        r = requests.get(url)
    except:
        print('Stopped at page: ' + page_num)
        break
    soup = BeautifulSoup(r.content)
    images = soup.find_all('img')[1:]
    image_titles = [img['alt'] for img in images]
    image_files_src = [img['src'] for img in images]
    image_files_datasrc = [img.get('data-src', None) for img in images]
    image_files = [src if datasrc is None else datasrc for src, datasrc in zip(image_files_src, image_files_datasrc)]
    
    price_elements = soup.find_all('span', class_ = 's-item__price')
    prices = [parse_price(price_e) for price_e in price_elements]
    try:
        assert len(prices) == len(images)
    except:
        print('Found unequal number of prices in page_num % d' % page_num)
        prices = [None] * len(images)
        
    for i in range(len(images)):
        title, file_name = download_image(image_files[i], image_titles[i], './girls/' + str(all_items_counter + i) + '.jpg')
        girls_items_data['title'][all_items_counter + i] = title
        girls_items_data['file_id'][all_items_counter + i] = all_items_counter + i
        girls_items_data['price'][all_items_counter + i] = prices[i]
    all_items_counter += len(images)
    f.value += 1

IntProgress(value=0, max=400)

In [17]:
girls_items_data = pd.DataFrame(girls_items_data)
girls_items_data = girls_items_data[girls_items_data['title'] != '']
print(girls_items_data.shape)

(16678, 3)


In [18]:
girls_items_data = pd.DataFrame(girls_items_data).drop_duplicates('title')
print(girls_items_data.shape)

(14257, 3)


In [19]:
girls_items_data.to_csv('girls_items_data.csv', index_label = 'id')
print('Final girls number of images: %d' % girls_items_data.shape[0])

Final girls number of images: 14257


In [27]:
def sample_train_test(df, n_train, n_test):
    df_train = df.sample(n = n_train)
    file_id_diff = set(df.file_id).difference(df_train.file_id)
    df_test = df[df.file_id.isin(file_id_diff)].sample(n = n_test)
    return df_train, df_test

boys_train, boys_test = sample_train_test(boys_items_data, 10000, 2500)
girls_train, girls_test = sample_train_test(girls_items_data, 10000, 2500)

print('N train boys: %d, N test boys: %d' % (len(boys_train.index), len(boys_test.index)))
print('N train girls: %d, N test girls: %d' % (len(girls_train.index), len(girls_test.index)))

boys_train.to_csv('boys_train.csv', index_label = 'id')
boys_test.to_csv('boys_test.csv', index_label = 'id')
girls_train.to_csv('girls_train.csv', index_label = 'id')
girls_test.to_csv('girls_test.csv', index_label = 'id')

N train boys: 10000, N test boys: 2500
N train girls: 10000, N test girls: 2500
